In [38]:
import pandas as pd
import random
#replace with your direcotry 
df = pd.read_csv('/Users/fhamman/Desktop/Projects /Balanced_Audio/transcript_metadata.csv')

# Display the first 5 rows
df.head()


STUDENT ENGLISH PROF: S_LEP

S_AFAM: Indicates if the student is African-American

S_ASIAN: Indicates if the student is Asian

S_HISP: Indicates if the student is Hispanic

S_WHITE: Indicates if the student is Caucasian

S_RACE_OTHER: Indicates if the student has a race/ethnicity not captured in other indicators



NCTETID: Teacher ID

TEACHER IS MALE: MALE

HISP: Q7: Teacher is Hispanic, regardless of race

BLACK: Q7: Teacher is Black, not of Hispanic origin

WHITE: Q7: Teacher is White, not of Hispanic origin

ASIAN: Q7: Teacher is Asian/Pacific Islander, not of Hispanic origin

AMINDIAN: Q7: Teacher is American Indian/Alaskan Native, not of Hisp. origin

MULTIRACIAL: Q7: Teacher is bi/multiracial, not of Hisp. origin

RACEOTHER: Q7: Teacher is 'other' race/ethnicity

In [6]:
# # Taking relavant features 
# subset_df = df.loc[:, ['NCTETID','S_MALE_count', 'S_LEP', 'S_AFAM', 'S_ASIAN', 'S_HISP', 'S_WHITE', 'S_RACE_OTHER', 'MALE', 'HISP', 'BLACK', 'WHITE', 'ASIAN', 'AMINDIAN', 'MULTIRACIAL', 'RACEOTHER']]
# # Check for null values in the subset dataframe
# subset_df.isnull().sum()
# # Drop entries with null values 
# subset_df.dropna(inplace=True)


# #This is not really necessary, you can just work wit df 


In [14]:
def compute_stats(dataset ):
    #TODO: Come up with a simplified logic to combine attibutes
    #ex add an arguments attr_to_combine
    #attr_to_combine = [['S_AFAM', 'S_Female'], ["T_Black", "T_Male"]]
    #Need also to check that combined attributes are not mutually exclusive
    teacher_male = dataset['MALE'].sum()/len(dataset)
    teacher_female = 1- teacher_male  #aka not male 
    T_Gender =  {'T_Male':teacher_male, 'T_Female':teacher_female}


    dataset['COUNT'] = dataset['S_MALE_count'] # Assume S_MALE_count is the number of individuals in each classroom 
    Stud_total = dataset['COUNT'].sum()
    eng_prof = (dataset['S_LEP']* dataset['COUNT']).sum()/ Stud_total
    S_Eng = {"1":eng_prof, '0': 1-eng_prof}


    AFAM = (dataset['S_AFAM']* dataset['COUNT']).sum()/ Stud_total
    ASIAN = (dataset['S_ASIAN']* dataset['COUNT']).sum()/ Stud_total
    HISP = (dataset['S_HISP']* dataset['COUNT']).sum()/ Stud_total
    WHITE = (dataset['S_WHITE']* dataset['COUNT']).sum()/ Stud_total
    Other = (dataset['S_RACE_OTHER'] * dataset['COUNT']).sum()/ Stud_total
    S_Race = {'AFAM':AFAM , 'ASIAN':ASIAN , "HISP":HISP , 'WHITE':WHITE, "Others":Other}


    T_AA = dataset['BLACK'].sum()/len(dataset)
    T_ASIAN = dataset['ASIAN'].sum()/len(dataset)
    T_HISP = dataset['HISP'].sum()/len(dataset)
    T_WHITE = dataset['WHITE'].sum()/len(dataset)
    T_AMINDIAN = dataset['AMINDIAN'].sum()/len(dataset)
    T_MULTIRACIAL = dataset['MULTIRACIAL'].sum()/len(dataset)
    T_Other = dataset['RACEOTHER'].sum()/len(dataset)
    T_RACE = {'T_AFAM':T_AA , 'T_ASIAN':T_ASIAN , "T_HISP":T_HISP , 'T_WHITE':T_WHITE, "T_AMINDIAN":T_AMINDIAN, "T_MULTIRACIAL":T_MULTIRACIAL, "T_Other":T_Other}



    stats =  {'S_Eng_Prof':S_Eng, 'S_Race':S_Race, 'T_Gender':T_Gender,'T_Race': T_RACE}
    return stats


In [16]:
compute_stats(df)

{'S_Eng_Prof': {'1': 0.2032421501097154, '0': 0.7967578498902845},
 'S_Race': {'AFAM': 0.41172622941071535,
  'ASIAN': 0.07676755663911515,
  'HISP': 0.23074864720223584,
  'WHITE': 0.23993577927097579,
  'Others': 0.040821787476957826},
 'T_Gender': {'T_Male': 0.17302798982188294, 'T_Female': 0.8269720101781171},
 'T_Race': {'T_AFAM': 0.1825699745547074,
  'T_ASIAN': 0.019083969465648856,
  'T_HISP': 0.03625954198473282,
  'T_WHITE': 0.6876590330788804,
  'T_AMINDIAN': 0.0,
  'T_MULTIRACIAL': 0.00826972010178117,
  'T_Other': 0.031170483460559797}}

In [73]:

def initialize_subset(dataset, size=100):
    return dataset.sample(n=size)

def calculate_imbalance(current_stats, target_distributions):
    imbalance = 0
    #TODO: imbalance as a dict to allow for weighted sum of imbalances
    for key in target_distributions:
        for subkey in target_distributions[key]:
            imbalance += abs(current_stats[key][subkey] - target_distributions[key][subkey])
        
    return imbalance

def optimize_subset(dataset, target_distributions, iterations=100000, swap_size=1, size=100):
    current_subset = initialize_subset(dataset, size = size)
    for _ in range(iterations):
        new_subset = current_subset.copy()
        to_remove = new_subset.sample(n=swap_size)
        to_add = dataset.loc[~dataset.index.isin(new_subset.index)].sample(n=swap_size)
        new_subset = pd.concat([new_subset.drop(to_remove.index), to_add])

        
        current_stats = compute_stats(current_subset)
        new_stats = compute_stats(new_subset)
        
        if calculate_imbalance(new_stats, target_distributions) < calculate_imbalance(current_stats, target_distributions):
            print(f'loss = {calculate_imbalance(current_stats, target_distributions)}')
            current_subset = new_subset
    
    return current_subset


In [78]:
target_distributions = {
    'T_Gender': {'T_Male': 0.5, 'T_Female': 0.5},
    'T_Race': {'T_AFAM': 0.25, 'T_ASIAN': 0.25, 'T_HISP': 0.25, 'T_WHITE': 0.25},  # Removed 'T_Other'
    'S_Eng_Prof': {'1': 0.5, '0': 0.5},
    'S_Race': {'AFAM': 0.25, 'ASIAN': 0.25, "HISP": 0.25, 'WHITE': 0.25}  # Removed "Others"
}

balanced_subset = optimize_subset(df, target_distributions, size = 100)
final_stats = compute_stats(balanced_subset)
print(final_stats)



loss = 2.7044927536231884
loss = 2.6936390101892287
loss = 2.6618426103646833
loss = 2.6531175059952035
loss = 2.6395785440613024
loss = 2.633461538461539
loss = 2.631948866377231
loss = 2.6237481910274965
loss = 2.614582730342499
loss = 2.614105058365759
loss = 2.6106692531522797
loss = 2.5993795093795087
loss = 2.592640422467594
loss = 2.5886964886964887
loss = 2.5691146589259795
loss = 2.56588377723971
loss = 2.548171991734223
loss = 2.538209787980233
loss = 2.4936647546207777
loss = 2.469601085048668
loss = 2.4660127591706544
loss = 2.4645928753180666
loss = 2.4300461856983593
loss = 2.363244613434727
loss = 2.3574238838933588
loss = 2.334584319293264
loss = 2.3180961416824797
loss = 2.3101466602901324
loss = 2.2935814249363875
loss = 2.261676998887299
loss = 2.2545171487276754
loss = 2.2305736287914506
loss = 2.1940768746061754
loss = 2.17880277252678
loss = 2.1634576534576535
loss = 2.159788950962135
loss = 2.1554563768789707
loss = 2.122707558859975
loss = 2.0958155369854534
los

In [79]:
len(balanced_subset)

100

In [80]:
balanced_subset
#TODO: Dump to a xlsx or json file 

,OBSID,year,student_text,teacher_text,student_cleaned,teacher_cleaned,NCTETID,DISTRICT,CLASS_11,CLASS_12,...,SES10,SES10_count,SES11,SES11_count,SES12,SES12_count,TSP01,TSP01_count,num_chapters,COUNT
985,2467,2,"Good morning, Teacher. [Inaudible]. Yes, I did...","Good morning, everyone. Now, can I have everyo...",good morning teacher inaudible yes i did landm...,good morning everyone now can i have everyone ...,11023,11.0,1.0,1.0,...,0.000000,15.0,0.466667,15.0,0.066667,15.0,4.066667,15.0,10,18.0
1518,4414,3,"Two dimension is flat, and three dimension is ...","Okay, today we’re going to learn some basic ge...",two dimension is flat and three dimension is i...,okay today we re going to learn some basic geo...,11072,11.0,0.0,1.0,...,0.111111,18.0,0.277778,18.0,0.055556,18.0,4.647059,17.0,8,18.0
1033,2535,2,"Length, width, and height. Height. Graph paper...",Okay class today we’re gonna talk about volume...,length width and height height graph paper sta...,okay class today we re gonna talk about volume...,11074,11.0,0.0,1.0,...,0.050000,20.0,0.450000,20.0,0.000000,20.0,4.093750,32.0,4,23.0
460,789,1,"Yeah, we did that last time. [Multiple comment...","All right. Today in math, what we're gonna do...",yeah we did that last time multiple comments i...,all right today in math what we re gonna do we...,11053,11.0,1.0,1.0,...,NaN,0.0,NaN,0.0,NaN,0.0,4.250000,12.0,8,12.0
1092,2634,2,"Like two – Like to me, like if you have a – th...","Boys and girls, we’re going to start our lesso...",like two like to me like if you have a the rul...,boys and girls we re going to start our lesson...,11060,11.0,1.0,1.0,...,0.071429,14.0,0.428571,14.0,0.214286,14.0,4.357143,14.0,8,17.0
1229,2916,2,"We were talking about fractions. Well, basica...","So this is math lesson, March 26th, 2012, 11:3...",we were talking about fractions well basically...,so this is math lesson march 26th 2012 11 30 s...,11057,11.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
1179,2758,2,We learned about the [inaudible]. The angle [i...,This is the beginning of the math lesson on Fe...,we learned about the inaudible the angle inaud...,this is the beginning of the math lesson on fe...,11057,11.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
832,2158,2,[Inaudible]. [Inaudible]. Up and down. Four. T...,"Ok, guys, you are going to want to – may I get...",inaudible inaudible up and down four two two f...,ok guys you are going to want to may i get sta...,14117,14.0,0.0,1.0,...,0.181818,22.0,0.090909,22.0,0.045455,22.0,4.318182,22.0,5,25.0
1404,4106,3,Students will be able to compare decimals usin...,"Okay, my math goals reader, can you please rea...",students will be able to compare decimals usin...,okay my math goals reader can you please read ...,11056,11.0,1.0,1.0,...,0.133333,15.0,0.066667,15.0,0.133333,15.0,4.600000,15.0,7,14.0
814,2140,2,I don’t know. It’s blinking. Me. Me. Me. Me...,Ten seconds. You all go back to your seats. ...,i don t know it s blinking me me me me a parti...,ten seconds you all go back to your seats ten ...,14127,14.0,0.0,1.0,...,0.000000,21.0,0.190476,21.0,0.047619,21.0,4.380952,21.0,7,23.0
